In [ ]:
import geopandas as gpd
from geopandas.geoseries import *
import pandas as pd 
import numpy as np

# Get shapes of Berlin's Planungsräume (PLR)
# 448 PLRs
# source: Amt für Statistik Berlin-Brandenburg
# https://fbinter.stadt-berlin.de/fb/berlin/service_intern.jsp?id=s_lor_plan@senstadt&type=WFS
plr_geo = gpd.read_file('source/LOR_Planungsraeume_2019.geojson')

# Get additional data to Berlin's Planungsräume
# source see above (FIS-Broker)
plr_data = pd.read_csv('source/LOR_Planungsraeume_2019.csv', sep=';')
plr_data.rename(columns={'Planungsraum': 'PLANUNGSRAUM', 'Bezirksregion': 'BEZIRKSREGION'}, inplace=True)

plr = plr_geo.merge(plr_data, on=['PLANUNGSRAUM', 'BEZIRKSREGION'])
plr.drop(['DATUM_GUELTIG_AB', 'Flächengröße in m²'], axis=1, inplace=True)
plr.rename(columns={'Schlüssel': 'SCHLUESSEL'}, inplace=True)
plr['SCHLUESSEL'] = plr['SCHLUESSEL'].astype(str)
plr.insert(0, 'New_ID', range(0, 0 + len(plr)))

plr_geo = plr.copy()
plr_geo.drop(['BEZIRKSNAME', 'BEZIRKSREGION', 'FLAECHENGROESSE_IN_M2', 'PROGNOSERAUM'], axis=1, inplace=True)
plr_geo.rename(columns={'PLANUNGSRAUM': 'PLR_NAME'}, inplace=True)
plr_geo.to_file('output/planungsraeume.geojson', driver='GeoJSON')


# buffer and projection in meter, see https://epsg.io/5678
projection = "EPSG:5678"
# buffer in meter (3 km/h walking pace; what is reachable in 10 min if your start at the boundary of PLR) => 500 m
bufferSize = 500 
plr = plr.to_crs(projection)


# source: Geoportal Berlin/Verwaltungseinheiten der Berliner Forsten
# https://fbinter.stadt-berlin.de/fb/index.jsp?loginkey=zoomStart&mapId=wmsk_forst_verwalt2014@senstadt&bbox=325121,5781695,451821,5848055
forests = gpd.read_file('source/waldflaechen.geojson')
forests.insert(0, 'New_ID', range(0, 0 + len(forests)))
forests.to_file("output/waldflaechen.geojson", driver="GeoJSON")
forests = forests.to_crs(projection)
forests["area_text"] = (forests["area_text"].replace(',', '.', regex=True).astype(float))

plr['area'] = None
plr['bufferedGeometry'] = None
plr['bufferedArea'] = None
plr['bufferedArea'] = None

bufferedGeometry = plr.geometry.to_crs(projection)
copyGeometry = bufferedGeometry
bufferedGeometry = bufferedGeometry.buffer(bufferSize)

for index, row in plr.iterrows():
    plr.loc[index, 'area'] = copyGeometry[index].area / 10000
    plr.loc[index, 'bufferedArea'] = bufferedGeometry[index].area / 10000

copyGeometry = bufferedGeometry.to_crs("EPSG:4326")
plr['bufferedGeometry'] = bufferedGeometry


# https://fbinter.stadt-berlin.de/fb/berlin/service_intern.jsp?id=s_gruenanlagenbestand@senstadt&type=WFS
parks = gpd.read_file('source/gruenflaechen.geojson')

# filter data (some parks have no data)
parks = parks.loc[parks['KATASTERFL'] > 0]

parks.insert(0, 'New_ID', range(len(forests), len(forests) + len(parks)))
parks.to_file('output/gruenflaechen.geojson', driver='GeoJSON')

parks = parks.to_crs(projection)

# calc area with geodata
parks['AREA'] = None
for index, row in parks.iterrows():
    parks.loc[index, 'AREA'] = row.geometry.area
    
print(parks.AREA.sum()) # 60890963.18207572 calculated data using geometries
print(parks['KATASTERFL'].sum()) # 61475596.6 
print(parks.count()) # 2518

bufferGeometry = plr.copy()
bufferGeometry = bufferGeometry.drop(columns="geometry")
bufferGeometry = bufferGeometry.drop(columns="bufferedGeometry")
bufferGeometry = bufferGeometry[['New_ID', 'SCHLUESSEL', 'PLANUNGSRAUM']]
bufferGeometry.rename(columns = {'PLANUNGSRAUM': 'PLR_NAME'}, inplace = True)

bufferGeometry['New_ID'] = bufferGeometry['New_ID'].astype(str)
bufferGeometry['SCHLUESSEL'] = bufferGeometry['SCHLUESSEL'].astype(str)

gdf = gpd.GeoDataFrame(bufferGeometry, geometry=copyGeometry)
gdf.to_file('output/buffer.geojson', driver='GeoJSON')


# initialize the following columns with empty lists
plr['openSpacesInBufferedPLR'] = np.empty((len(plr), 0)).tolist()
plr['openSpacesInPLR'] = np.empty((len(plr), 0)).tolist()
plr['allOpenSpacesArea'] = np.empty((len(plr), 0)).tolist()
plr['allOpenSpacesBufferedArea'] = np.empty((len(plr), 0)).tolist()


# interate over PLRs and check which park or forests intersects with the corresponding plr geometry resp. buffer
for indexPLR, rowPLR in plr.iterrows():
    for indexOS, rowOS in parks.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(str(rowOS.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesBufferedArea'].append(rowOS.KATASTERFL)
        if rowPLR.geometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(str(rowOS.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesArea'].append(rowOS.KATASTERFL)
        if rowPLR.SCHLUESSEL.startswith('0'):
            plr.loc[indexPLR, 'SCHLUESSEL'] = rowPLR.SCHLUESSEL[1:]
    for indexF, rowF in forests.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowF.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(str(rowF.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesBufferedArea'].append(rowF.area_text)
        if rowPLR.geometry.intersects(rowF.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(str(rowF.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesArea'].append(rowF.area_text)

plr['bufferedGeometry'] = copyGeometry

print(plr['area'].sum()) # sum of all PLRs => 89132.31009939541 hectar

plr['numberOfSpacesInPLR'] = None
plr['numberOfSpacesInBufferedPLR'] = None

for index, row in plr.iterrows():
    plr.loc[index, 'numberOfSpacesInPLR'] = len(row.openSpacesInPLR)
    plr.loc[index, 'numberOfSpacesInBufferedPLR'] = len(row.openSpacesInBufferedPLR)

    
# merge data with population figures

# Amt für Statistik Berlin-Brandenburg: Einwohnerinnen und Einwohner im Land Berlin LOR-Planungsräume
# (https://www.statistik-berlin-brandenburg.de/Statistiken/statistik_SB.asp?Ptyp=700&Sageb=12041&creg=BBB&anzwer=10)
data = pd.read_csv('source/einwohner.csv')

data.rename(columns = {'RAUMID': 'SCHLUESSEL'}, inplace = True)
data = data[['SCHLUESSEL', 'E_E']]
data['E_E'] = data['E_E'].astype(str)

data['SCHLUESSEL'] = data['SCHLUESSEL'].astype(str)
plr['SCHLUESSEL'] = plr['SCHLUESSEL'].astype(str)

output = pd.merge(plr, data, on='SCHLUESSEL', how='left')

output = output.rename(columns={'E_E': 'population'})

for index, row in output.iterrows():
    output.loc[index, 'allOpenSpacesBufferedArea'] = sum(row.allOpenSpacesBufferedArea) / 10000
    output.loc[index, 'allOpenSpacesArea'] = sum(row.allOpenSpacesArea) / 10000

# reproject to WGS84 (geometry in degrees)
output = output.to_crs('EPSG:4326')


# merge data with PLZs

PLZ = pd.read_csv('output/PLR2PLZ.csv')
output['SCHLUESSEL'] = output['SCHLUESSEL'].astype(str)
PLZ['SCHLUESSEL'] = PLZ['PLR'].astype(str)

PLZ = PLZ[['SCHLUESSEL', 'PLZ']]

output = pd.merge(PLZ, output, on='SCHLUESSEL')

output.rename(columns={'BEZIRKSREGION': 'BZRNAME', 'PLANUNGSRAUM': 'PLR_NAME', 'BEZIRKSNAME': 'BEZNAME', 'PROGNOSERAUM': 'PGRNAME'}, inplace=True)
output = output[['New_ID', 'SCHLUESSEL', 'PLR_NAME', 'BZRNAME', 'PGRNAME', 'BEZNAME', 'PLZ', 'geometry', 'area', 'bufferedGeometry', 'bufferedArea', 'openSpacesInBufferedPLR', 'openSpacesInPLR', 'allOpenSpacesArea', 'allOpenSpacesBufferedArea', 'numberOfSpacesInPLR', 'numberOfSpacesInBufferedPLR', 'population']]

output.astype({'geometry': str}, {'bufferedGeometry': str}).to_csv('output/data.csv', sep=';')